# Capstone Project: The Battle of Neighborhoods (Week 4) Project #1

In [2]:
import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
%matplotlib inline 
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
mpl.style.use('ggplot') # optional: for ggplot-like style

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

ModuleNotFoundError: No module named 'geopy'

In [4]:
!pip install geopy

  Using cached geographiclib-1.50-py3-none-any.whl (38 kB)


You should consider upgrading via the 'c:\programdata\anaconda3\python.exe -m pip install --upgrade pip' command.


In [3]:
import pandas as pd
import numpy as np

### I chose a Project to give simulations for deciding on which type of business to be launched based on the most common venues near the Top Hotels in Paris. Based on the data which you get from Foursquare I tried to see the most common venues and make a list of possible kinds of businesses that can be opened near these venues. 

### A business will be successful only if in a particular location there is good demand of a particular need, good number of customeers and less number of supplies.

### For which I've started to pull the data of the highest possible crowd in Paris that could be either the famous monuments or could be the Hotels where people reside for more days for a business requirement or could be a tour etc.

### Gathering the most common venues found near the top hotels in Paris and understanding the essentials in the view of customers or feedback can outcome with a best suggestion for launching a business.

### Here, I tried to gather the top hotels and out of which I categorised the 20 most common venues near these venue and conclude the list of possible businesses that could be launched.

# 1 : Hotel_data in Paris

In [ ]:
Paris_Hotel_data = pd.read_csv("D:/Work/Paris_Hotel_data5.csv",index_col = 0)
Paris_Hotel_data.head()

In [ ]:
# Paris_Hotel_data = Paris_Hotel_data.dropna(how="any", inplace=True)
Paris_Hotel_data = Paris_Hotel_data.mask(Paris_Hotel_data.eq('None')).dropna()
print(Paris_Hotel_data)

# Localise Hotel in the Map

In [ ]:
# Localise places on the map Paris
address = 'Paris'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Paris are {}, {}.'.format(latitude, longitude))

In [ ]:
# Paris_Hotel_data_dropna = Paris_Hotel_data.reset_index(drop=True)
# # toronto_df_dropna.head()
# Paris_Hotel_data_dropna.head()

In [ ]:
#create map of Paris using latitude and longitude values
map_Paris = folium.Map(location=[latitude, longitude], zoom_start=14)

# add markers to map
for lat, lng, label in zip(Paris_Hotel_data['lat'], Paris_Hotel_data['lng'], Paris_Hotel_data['Hotel_Name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='red',
        fill=True,
        fill_color='white',
        fill_opacity=0.7
    ).add_to(map_Paris)  
    
map_Paris

# Define Foursquare Credentials and Version

In [ ]:
#Define Foursquare Credentials and Version
CLIENT_ID = 'D0MXEX5Y2KEIYEKEJMXBKTV5T4DSZGIIB52UPAZXGGIZ3J1Z' # your Foursquare ID
CLIENT_SECRET = 'T2YE2IKXSGGPYNRYJO2X1IDFNUT23P4MWGIVAS1TCOZ1IEIV' # your Foursquare Secret
VERSION = '20180604'
#LIMIT = 30

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [ ]:
from pandas.io.json import json_normalize

# Explore all venue's categories in Paris

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng,            
            v['venue']['name'], 
            v['venue']['id'],
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Places', 
                  'Places Latitude', 
                  'Places Longitude', 
                  'Venue',
                  'Venue id',                
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category'        
                   ]
    
    return(nearby_venues)

In [ ]:
Paris_Hotel_data_venues = getNearbyVenues(names=Paris_Hotel_data['Hotel_Name'],
                                   latitudes=Paris_Hotel_data['lat'],longitudes=Paris_Hotel_data['lng'])

In [ ]:
print(Paris_Hotel_data_venues.shape)
Paris_Hotel_data_venues.to_csv('D:/Work/file1.csv') 

In [ ]:
#Let's check how many venues were returned for each lieux
Paris_Hotel_data_venues.groupby('Places').count()

In [ ]:
Paris_Hotel_data_venues1 = Paris_Hotel_data_venues.groupby('Venue Category')
Paris_Hotel_data_venues1.head()

# Number by Categories at these Places

In [ ]:
df_Hview = Paris_Hotel_data_venues.groupby('Places').count()
df_Hotel = df_Hview['Venue Category']
df_Hotel

In [ ]:
df_Hotel.plot(kind='barh', figsize=(10,6))

#plt.xlabel('name') # add to x-label to the plot
#plt.ylabel('Number de categories') # add y-label to the plot
plt.title('Number Categories\' Venue by Places') # add title to the plot

for index, value in enumerate(df_Hotel): 
    label = format(int(value), ',') 
    plt.annotate(label, xy=(value - 10, index - 0.10), color='white')
    
plt.show()

In [ ]:
#Let's find out how many unique categories can be curated from all the returned venues¶
print('There are {} uniques categories.'.format(len(Paris_Hotel_data_venues['Venue Category'].unique())))
print(Paris_Hotel_data_venues['Venue Category'].unique() )

# Another Analyses for each venue

In [ ]:
# one hot encoding
paris_onehot = pd.get_dummies(Paris_Hotel_data_venues[['Venue Category']], prefix="", prefix_sep="")

# add Lieux column back to dataframe
paris_onehot['Places'] = Paris_Hotel_data_venues['Places'] 

# move lieux column to the first column
fixed_columns = [paris_onehot.columns[-1]] + list(paris_onehot.columns[:-1])
paris_onehot = paris_onehot[fixed_columns]

paris_onehot.head()

In [ ]:
paris_onehot.shape

In [ ]:
# Next, 
# let's group rows by venue
# and by taking the mean of the frequency of occurrence of each category
paris_grouped = paris_onehot.groupby('Places').mean().reset_index()
paris_grouped

In [ ]:
#Let's print each venue along with the top 20 most common venues¶
num_top_venues = 20

for hood in paris_grouped['Places']:
    print("----"+hood+"----")
    temp = paris_grouped[paris_grouped['Places'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

In [ ]:
#Let's put that into a pandas dataframe
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
#Now let's create the new dataframe 
#and display the top 20 venues for each.
num_top_venues = 20

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Places']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
lieux_venues_sorted = pd.DataFrame(columns=columns)
lieux_venues_sorted['Places'] = paris_grouped['Places']

for ind in np.arange(paris_grouped.shape[0]):
    lieux_venues_sorted.iloc[ind, 1:] = return_most_common_venues(paris_grouped.iloc[ind, :], num_top_venues)

lieux_venues_sorted

In [ ]:
# set number of clusters
kclusters = 3

paris_grouped_clustering = paris_grouped.drop('Places', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(paris_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

In [ ]:
#Let's create a new dataframe that includes the cluster as well as the top 5 venues for each Lieux.

paris_merged = Paris_Hotel_data

# add clustering labels
paris_merged['Cluster Labels'] = kmeans.labels_[0:10]

paris_merged = paris_merged.join(lieux_venues_sorted.set_index('Places'), on='Hotel_Name')
paris_merged 
# check the last columns!

In [ ]:

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=14)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(paris_merged['lat'], paris_merged['lng'], paris_merged['Hotel_Name'], paris_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
# Cluster 0 == Hotel L Antonio,H tel Victor Hugo Paris Kl ber Cluster
paris_merged.loc[paris_merged['Cluster Labels'] == 0, paris_merged.columns[[1] + list(range(5, paris_merged.shape[1]))]]

In [ ]:
# Cluster 1 == H tel Westminster
paris_merged.loc[paris_merged['Cluster Labels'] == 1, paris_merged.columns[[1] + list(range(5, paris_merged.shape[1]))]]

In [ ]:
# Cluster 2 == Novotel Paris Gare De Lyon,Mercure Paris Pigalle Sacre Coeur,H tel Diva Opera
paris_merged.loc[paris_merged['Cluster Labels'] == 2, paris_merged.columns[[1] + list(range(5, paris_merged.shape[1]))]]

### The most common business we find near the best hotels is the food business. Number of French, Italian restaurants could be found here that includes the coffee bar, resto bar, Gym, Music Venue etc.
### Apart from the food related businesses, we have other types of business that maybe found throughout the Paris but not near these venues
### so the list of possible ideas could be the '1. Travel and Guide', '2. Gifts/ Cultural Shops', '3. Taxi Businesses', '4. Jewelleries and Accessories Shops', '5. Photography', '6. Unique Imported Wines Business', '7. Sell Beauty and Wellness Products', '8. Natural Food Business' etc..